In [47]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from geopy.geocoders import Nominatim
import math
import time
import csv


In [48]:
df_aed = pd.read_parquet('aed_locations.parquet.gzip')

In [49]:
df_aed.loc[df_aed['id']==15877.0,]

,id,type,address,number,postal_code,municipality,province,location,public,available,hours
14089,15877.0,Appareil fixe-Vast apparaat,Vaartkant Rechts,NaN,2960.0,Brecht,Antwerpen,Binnenkomst via zwarte poort -> ga naar de laa...,Non-Nee,Non-Nee,None
14098,15877.0,Appareil fixe-Vast apparaat,Van Immerseelstraat,66.0,2000.0,Anvers,Antwerpen,reception,Oui-Ja,Non-Nee,None
14176,15877.0,Appareil fixe-Vast apparaat,Smalle Heerweg,31.0,9041.0,OOSTAKKER,Oost-Vlaanderen,"Gebouw BTJ, in de inkom.",Oui-Ja,Oui-Ja,None


In [42]:
df_aed['id'].value_counts()

id
15877.0    3
15881.0    3
15879.0    3
15878.0    3
15882.0    3
          ..
6095.0     1
6096.0     1
6097.0     1
6098.0     1
16668.0    1
Name: count, Length: 14907, dtype: int64

In [72]:
df_aed['addresses']= None
df_aed['lat'] = None
df_aed['lon'] = None

In [79]:
df_aed.loc[0:2,]

,id,location,public,available,hours,addresses,lat,lon
0,13.0,None,Y,None,None,"Blvd. Fr. Roosevelt 24 ,7060 ,SOIGNIES , Belgium",None,None
1,70.0,None,None,None,None,"Ch. De Wégimont 76 ,4630 ,Ayeneux , Belgium",None,None
2,71.0,None,None,None,None,"Place Saint - Lambert ,4020 ,Liège , Belgium",None,None


In [73]:
#create address as one string 
number = 1
for number in range(len(df_aed)):
    address = ''
    if not pd.isna(df_aed['address'][number]):
        address = str(df_aed['address'][number])
        if not pd.isna(df_aed['number'][number]):
            address = address+" "+str(int(df_aed['number'][number]))
        if not pd.isna(df_aed['postal_code'][number]):
            address = address +" ,"+str(int(df_aed['postal_code'][number]))
        if not pd.isna(df_aed['municipality'][number]):
            address = address +" ,"+str(df_aed['municipality'][number])
        address = address + ' , Belgium'
        df_aed.loc[number,'addresses'] = address

In [ ]:
df_aed.drop(['type','address','number','postal_code','municipality','province','public'], axis=1,inplace=True)

In [81]:
geolocator = Nominatim(user_agent="isabel")

In [82]:
counter = 0
not_found = 0
for address in range(len(df_aed)):
    location=geolocator.geocode(address, timeout=15)
    if location == None:
        not_found += 1
        if (not_found % 10 == 0):
            print('number of not found addresses:'+ str(not_found))
            time.sleep(5)
    else:
        df_aed.loc[counter,'lon'] =location.longitude
        df_aed.loc[counter,'lat'] =location.latitude
    counter += 1
    if (counter % 20 ==0):
        print('sleeping....')
        time.sleep(5)
        print('awake and count is: '+str(counter))


sleeping....
awake and count is: 20
sleeping....
awake and count is: 40
sleeping....
awake and count is: 60
sleeping....
awake and count is: 80
sleeping....
awake and count is: 100
sleeping....
awake and count is: 120
sleeping....
awake and count is: 140
sleeping....
awake and count is: 160
sleeping....
awake and count is: 180
sleeping....
awake and count is: 200
sleeping....
awake and count is: 220
sleeping....
awake and count is: 240
sleeping....
awake and count is: 260
sleeping....
awake and count is: 280
sleeping....
awake and count is: 300
sleeping....
awake and count is: 320
sleeping....
awake and count is: 340
sleeping....
awake and count is: 360
sleeping....
awake and count is: 380
sleeping....
awake and count is: 400
sleeping....
awake and count is: 420
sleeping....
awake and count is: 440
sleeping....
awake and count is: 460
sleeping....
awake and count is: 480
sleeping....
awake and count is: 500
sleeping....
awake and count is: 520
sleeping....
awake and count is: 540
sleep

In [97]:
df_aed.loc[df_aed['lat'].isnull()]

,id,location,public,available,hours,addresses,lat,lon
1,70.0,None,None,None,None,"Ch. De Wégimont 76 ,4630 ,Ayeneux , Belgium",None,None
2,71.0,None,None,None,None,"Place Saint - Lambert ,4020 ,Liège , Belgium",None,None
3,72.0,None,None,None,None,"Rue du Doyard ,4990 ,Lierneux , Belgium",None,None
4,73.0,None,None,None,None,"Fond Saint Servais ,4000 ,Liège , Belgium",None,None
5,74.0,None,None,None,None,"Rue des Prémontrès 12 ,4020 ,Liège , Belgium",None,None


In [83]:
file_path = 'aed_coordinates.csv'

In [84]:
# Write the list to a CSV file
df_aed.to_csv(file_path, index=False)
print("CSV file saved successfully.")

CSV file saved successfully.


handle the aed addresses that were not read the first time, use a different geolocator


In [9]:
some_locations = gpd.tools.geocode(leftover_list[1])

In [22]:
counter = 0
coords = pd.DataFrame(columns=['geometry', 'address'])

for i in range(len(leftover_list)):
    some_locations = gpd.tools.geocode(leftover_list[counter: counter+20],timeout = 10)
    coords= pd.concat([coords, some_locations])
    counter += 20
    time.sleep(5)
    print('awake again....')
    print(counter)

C:\Users\isa_s\AppData\Local\Temp\ipykernel_1112\3589298492.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coords= pd.concat([coords, some_locations])


awake again....
20
awake again....
40
awake again....
60
awake again....
80
awake again....
100
awake again....
120
awake again....
140
awake again....
160
awake again....
180
awake again....
200
awake again....
220
awake again....
240
awake again....
260
awake again....
280
awake again....
300
awake again....
320
awake again....
340
awake again....
360
awake again....
380
awake again....
400
awake again....
420
awake again....
440
awake again....
460
awake again....
480
awake again....
500
awake again....
520
awake again....
540
awake again....
560
awake again....
580
awake again....
600
awake again....
620
awake again....
640
awake again....
660
awake again....
680
awake again....
700
awake again....
720
awake again....
740
awake again....
760
awake again....
780
awake again....
800
awake again....
820
awake again....
840
awake again....
860
awake again....
880
awake again....
900
awake again....
920
awake again....
940
awake again....
960
awake again....
980
awake again....
1000
awa

ValueError: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Supply geometry using the 'geometry=' keyword argument, or by providing a DataFrame with column name 'geometry'

In [23]:
file_path = 'aed_addresses_coords2.csv'
leftover = pd.DataFrame(coords)
# Write the list to a CSV file
leftover.to_csv(file_path, index=False)
print("CSV file saved successfully.")

CSV file saved successfully.


In [24]:
coords_read = pd.read_csv(file_path)

In [25]:
coords_read.head()

,geometry,address
0,POINT (12.0133335 10.5131945),"UTC+01:00 standard time, Nigeria"
1,POINT (4.0656826 50.5771174),"Octopus-Soignies asbl, 24, Boulevard Franklin ..."
2,POINT (5.737994592222156 50.6124391),"Château de Wégimont, 76, Chaussée de Wégimont,..."
3,POINT (5.665886999165581 50.44114105),"1, Rue de Bastogne, 4920, Rue de Bastogne, Ayw..."
4,POINT (5.583379851103196 50.64334765),"19, Boulevard de la Constitution, 4020, Boulev..."


In [26]:
coords_read_nodups = coords_read.drop_duplicates()

In [36]:
addr_test = pd.DataFrame(addresses)

In [41]:
addr_test.value_counts().iloc[1:20]

0                                                    
Rue du Charroi  20 ,1190 ,Bruxelles , Belgium            47
Appareil nomade non attribué à un véhicule , Belgium     28
Avenue Porte de Hal  40 ,1060 ,Bruxelles , Belgium       19
Scheldelaan 460 ,2040 ,Antwerpen , Belgium               15
Rue Wiertz  60 ,1047 ,BRUXELLES , Belgium                15
Turnhoutseweg 30 ,2340 ,Beerse , Belgium                 15
Rue de Brigniot 205 ,5002 ,Saint-Servais , Belgium       14
rue des Frères Wright  8 ,6041 ,Gosselies , Belgium      14
Scheldelaan  460 ,2040 ,Antwerpen , Belgium              13
Bvd Léopold 285 ,1081 ,Bruxelles , Belgium               11
Battelsesteenweg 315 ,2800 ,Mechelen , Belgium           11
Noorderlaan 500 ,2030 ,Antwerpen , Belgium               10
Bvd du Jardin Botanique 15 ,1000 ,Bruxelles , Belgium    10
Industriepark 30 ,2220 ,Heist-Op-Den-Berg , Belgium      10
Chemin du Foriestl) ,1420 ,Braine-l Alleud , Belgium     10
Station Gare du Midi 1 ,1060 ,Bruxelles , Belg

In [31]:
coords_read['address'].value_counts()

address
Brussel Noord - Bruxelles Nord, Rue du Progrès - Vooruitgangstraat, 1030, Rue du Progrès - Vooruitgangstraat, Schaerbeek - Schaarbeek, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, België / Belgique / Belgien    15
205, Rue de Bricgniot, 5002, Rue de Bricgniot, Saint-Servais, Namur, België / Belgique / Belgien                                                                                                                                     14
Boulevard Léopold II - Leopold II-laan, 1081, Koekelberg, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, België / Belgique / Belgien                                                                                 12
Bruxelles-Midi - Brussel-Zuid, Galerie M - M-galerij, 1060, Galerie M - M-galerij, Saint-Gilles - Sint-Gillis, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, België / Belgique / Belgien                            11
North Light, 34, Boulevard Simon Bolivar - Simon Bolivarlaan, 10

In [30]:
coords_read.

,geometry,address
43,POINT EMPTY,NaN
44,POINT EMPTY,NaN
119,POINT EMPTY,NaN
132,POINT EMPTY,NaN
143,POINT EMPTY,NaN
...,...,...
2798,POINT EMPTY,NaN
2799,POINT EMPTY,NaN
2800,POINT EMPTY,NaN
2806,POINT EMPTY,NaN


there are still 600 geolocations missing....

In [126]:
column_names = ['lon', 'lat']
file_path = 'lefover_coords.csv'
# Read the CSV file into a pandas DataFrame
coords_cleaned = pd.read_csv(file_path, names = column_names)

In [127]:
all_coordinates= pd.concat([aed_coordinates, coords_cleaned])
print(len(all_coordinates))

14640


In [124]:
px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 7       # Zoom level (1-20)
# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
aed_location_layer = px.scatter_mapbox(all_coordinates, lat='lat', lon='lon')

# Add the scatter plot layer to the base map
map_with_aed_points = map_base.add_trace(aed_location_layer.data[0])

# Show the map
map_with_aed_points.show()